# Two Sigma: Rental Interest Competition

In [1]:
import datetime
import itertools
import json
import operator
import os
import pandas as pd
import pprint
import numpy as np
import re
import spacy
import torch

from collections import Counter, deque
from sklearn.metrics import mean_squared_error

from utils.data import load_training_test_data
from utils.dataframe import (
    categories_from_column,
    column_list_to_category_flags,
    count_ngrams_up_to_n,
    map_categorical_column_to_category_ids,
    normalize_categories,
    remap_column,
    remap_date_column_to_days_before,
    remove_small_or_stopwords_from_ranking
)
from utils.doc2vec import (
    column_to_doc_vectors
)
from utils.model import (
    basic_logistic_regression_pipeline,
    format_statistics,
    get_prediction_probabilities_with_columns,
    prediction_accuracy,
    test_model_with_k_fold_cross_validation
)

nlp = spacy.load("en")

Check GPU support

In [2]:
torch.cuda.is_available()

True

## 1) Load data

In [3]:
(TRAIN_DATAFRAME, TEST_DATAFRAME) = \
  load_training_test_data(os.path.join('data', 'train.json'),
                          os.path.join('data', 'test.json'))

## 2) Data Cleaning

Let's see what this table looks like. We'll display the head of the table which shows its features

In [4]:
TRAIN_DATAFRAME.head()

,id,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level
0,4,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,-73.9539,a10db4590843d78c784171a107bdacb4,[https://photos.renthop.com/2/7170325_3bb5ac84...,2400,145 Borinquen Place,medium
1,6,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,-73.9722,955db33477af4f40004820b4aed804a0,[https://photos.renthop.com/2/7092344_7663c19a...,3800,230 East 44th,low
2,9,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,-73.9625,c8b10a317b766204f08e613cef4ce7a0,[https://photos.renthop.com/2/7158677_c897a134...,3495,405 East 56th Street,medium
3,10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,medium
4,15,1.0,0,bfb9405149bfff42a92980b594c28234,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,-73.9743,2c3b41f588fbb5234d8a1e885a436cfa,[https://photos.renthop.com/2/7225292_901f1984...,2795,340 East 34th Street,low


### 2.1) Cleaning up categories

Let's clean up the categories and put them into a sensible vector. Unfortunately the categories are a bit of a mess - since the user can specify what categories they want there isn't much in the way of consistency between categories.

Some of the patterns that we frequently see in the categories are:
 - Separating category names with "**"
 - Mix of caps/nocaps
 - Some common themes, such as:
   - "pets"
   - "office"
   - "living room"
   - "garden"
   - "common area"
   - "storage"
   - "no pets"
   - "parking"
   - "bicycle"
   - "doorman"
   - etc

To deal with this, lets pull out all of the categories and normalize them
by removing excess punctuation, normalizing for whitespace, lowercasing, and counting for certain n-grams.

In [5]:
normalized_categories = sorted(normalize_categories(categories_from_column(TRAIN_DATAFRAME, 'features')))
normalized_categories[:50]

['$1000. move in visa giftcard will be handed to new tenants upon rental lease signing',
 '$1500. move in visa giftcard will be handed to new tenants upon rental lease signing',
 '$250 application fee with background and credit check. board interview and approval.',
 '$250 security deposit',
 '$500 security deposit',
 '$600 gift card for move ins prior to june 19',
 '000 sf fitness center',
 '000 sq. ft. landscaped terrace complete with an organic garden',
 '000 square foot exclusive courtyard oasis for via residents',
 '000 square foot sun deck• free wifi in marc club',
 '1 br or conv 2',
 "1 mo broker's fee 18 mo lease",
 '1 month free',
 '1 month free',
 '1 month free',
 '1 month free rent',
 '1.5 bath',
 '1.5 bathrooms',
 '1.5 baths',
 '1.5 baths',
 '1.5 marble baths',
 '1/2 bath',
 '1/2 month fee',
 '1100 sq ft',
 '1200sq ft',
 '12th st 3rd ave',
 '1st',
 '2',
 '2 bathrooms',
 '2 beautifully furnished outside sundecks',
 '2 bedroon convertible',
 '2 blocks away from union square',

Now that we have our slightly tidied up categories, we can create some n-grams and count their frequency

In [6]:
most_common_ngrams = sorted(count_ngrams_up_to_n(" ".join(normalized_categories), 3).most_common(),
                            key=lambda x: (-x[1], x[0]))
most_common_ngrams[:50]

[('', 6421),
 ('in', 122),
 ('kitchen', 121),
 ('to', 90),
 ('room', 85),
 ('home', 65),
 ('of', 63),
 ('fee', 62),
 ('and', 59),
 ('no', 57),
 ('the', 57),
 ('private', 56),
 ('super', 56),
 ('2br', 54),
 ('new', 50),
 ('on', 49),
 ('2', 48),
 ('massive', 48),
 ('windows', 45),
 ('share', 44),
 ('sprawling', 44),
 ('no fee', 42),
 ('park', 42),
 ('storage', 42),
 ('pets', 41),
 ('ok', 40),
 ('bldg', 39),
 ('deck', 38),
 ('l', 37),
 ('laundry', 37),
 ('all', 35),
 ('huge', 33),
 ('eat', 32),
 ('eat in', 32),
 ('in kitchen', 32),
 ('oversized', 32),
 ('parking', 32),
 ('renovated', 32),
 ('roof', 32),
 ('3br', 31),
 ('blks', 31),
 ('blks to', 31),
 ('free', 31),
 ('on site', 30),
 ('site', 30),
 ('doorman', 29),
 ('super share', 29),
 ('with', 29),
 ('1br', 28),
 ('closets', 28)]

There's quite a few words here that don't add much value. We can remove them by consulting a list of stopwords

In [7]:
most_common_ngrams = sorted(list(remove_small_or_stopwords_from_ranking(most_common_ngrams, nlp, 3)),
                            key=lambda x: (-x[1], x[0]))
most_common_ngrams[:50]

[('kitchen', 121),
 ('room', 85),
 ('home', 65),
 ('fee', 62),
 ('private', 56),
 ('super', 56),
 ('2br', 54),
 ('new', 50),
 ('massive', 48),
 ('windows', 45),
 ('share', 44),
 ('sprawling', 44),
 ('no fee', 42),
 ('park', 42),
 ('storage', 42),
 ('pets', 41),
 ('bldg', 39),
 ('deck', 38),
 ('laundry', 37),
 ('huge', 33),
 ('eat', 32),
 ('eat in', 32),
 ('in kitchen', 32),
 ('oversized', 32),
 ('parking', 32),
 ('renovated', 32),
 ('roof', 32),
 ('3br', 31),
 ('blks', 31),
 ('blks to', 31),
 ('free', 31),
 ('on site', 30),
 ('site', 30),
 ('doorman', 29),
 ('super share', 29),
 ('1br', 28),
 ('closets', 28),
 ('included', 27),
 ('large', 27),
 ('steps', 27),
 ('studio', 27),
 ('bedford', 26),
 ('bedford l', 26),
 ('floor', 26),
 ('lounge', 26),
 ('pets ok', 26),
 ('steps to', 26),
 ('clean', 25),
 ('terrace', 25),
 ('train', 25)]

Now that we have these, we can probably take 100 most common and arrange
them into category flags for our table

In [8]:
TRAIN_DATAFRAME = column_list_to_category_flags(TRAIN_DATAFRAME, 'features', list(map(operator.itemgetter(0), most_common_ngrams[:100])))
TEST_DATAFRAME = column_list_to_category_flags(TEST_DATAFRAME, 'features', list(map(operator.itemgetter(0), most_common_ngrams[:100])))

In [9]:
TRAIN_DATAFRAME.head(5)

,id,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,...,features_in_unit,features_l_stop,features_stop,features_true,features_views,features_details,features_dryer,features_elev/lndry,features_elev/lndry_bldg,features_gut
0,4,1.0,1,8579a0b0d54db803821a35a4a615e97a,2016-06-16 05:55:27,Spacious 1 Bedroom 1 Bathroom in Williamsburg!...,145 Borinquen Place,"[Dining Room, Pre-War, Laundry in Building, Di...",40.7108,7170325,...,0,0,0,0,0,0,0,0,0,0
1,6,1.0,2,b8e75fc949a6cd8225b455648a951712,2016-06-01 05:44:33,BRAND NEW GUT RENOVATED TRUE 2 BEDROOMFind you...,East 44th,"[Doorman, Elevator, Laundry in Building, Dishw...",40.7513,7092344,...,0,0,0,0,0,0,0,0,0,0
2,9,1.0,2,cd759a988b8f23924b5a2058d5ab2b49,2016-06-14 15:19:59,**FLEX 2 BEDROOM WITH FULL PRESSURIZED WALL**L...,East 56th Street,"[Doorman, Elevator, Laundry in Building, Laund...",40.7575,7158677,...,1,0,0,0,0,0,0,0,0,0
3,10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,7211212,...,0,0,0,0,0,0,0,0,0,0
4,15,1.0,0,bfb9405149bfff42a92980b594c28234,2016-06-28 03:50:23,Over-sized Studio w abundant closets. Availabl...,East 34th Street,"[Doorman, Elevator, Fitness Center, Laundry in...",40.7439,7225292,...,0,0,0,0,0,0,0,0,0,0


### 2.2) Cleaning up listing_date

We can also do something useful with the listing date - it may be better to say how many days ago the property was listed - older properties are probably going to get a lot less interest than newer properties.

In [10]:
TRAIN_DATAFRAME = remap_date_column_to_days_before(TRAIN_DATAFRAME, "created", "created_days_ago", datetime.datetime(2017, 1, 1))
TEST_DATAFRAME = remap_date_column_to_days_before(TEST_DATAFRAME, "created", "created_days_ago", datetime.datetime(2017, 1, 1))

In [11]:
TRAIN_DATAFRAME["created_days_ago"].head(5)

0    198
1    213
2    200
3    190
4    186
Name: created_days_ago, dtype: int64

### 2.3) Cleaning up interest_level

Right now the interest level is encoded on a scale of "Low, Medium, High". The competition
wants us to classify the entries in to each, so we assign a label

In [12]:
TRAIN_DATAFRAME = remap_column(TRAIN_DATAFRAME, "interest_level", "label_interest_level", lambda x: {
    "high": 0,
    "medium": 1,
    "low": 2
}[x])
# The TEST_DATAFRAME does not have an interest_level column, so we
# instead add it and replace it with all zeros
TEST_DATAFRAME["label_interest_level"] = 0

In [13]:
TRAIN_DATAFRAME["label_interest_level"].head(5)

0    1
1    2
2    1
3    1
4    2
Name: label_interest_level, dtype: int64

### 2.4) Cleaning up building_id, manager_id

`building_id` and `manager_id` look a bit useless to us on the outside, but according to https://www.kaggle.com/den3b81/some-insights-on-building-id they are actually quite predictive of interest since 20% of the manager make up 80% of the rentals (we can also see this in their writing style as well).

Since there aren't too many managers or buildings in total, we can convert these into category ID's where we'll pass them through an embedding later on.

Note that we need to do this over both dataframes - since there could
be some managers that are in the test dataframe which are not in the training dataframe and vice versa.

Note that we want to lump all the "misc" buildings and managers together
into a single building or manager since listings by "non-property managers" or "non-frequently-rented-buildings" are different from ones run by property managers.

In [14]:
(BUILDING_ID_UNKNOWN_REMAPPING,
 BUILDING_ID_TO_BUILDING_CATEGORY,
 BUILDING_CATEGORY_TO_BUILDING_ID,
 TRAIN_DATAFRAME,
 TEST_DATAFRAME) = map_categorical_column_to_category_ids(
    TRAIN_DATAFRAME,
    TEST_DATAFRAME,
    'building_id',
    'building_id_category',
    min_freq=40
)

In [15]:
(MANAGER_ID_UNKNOWN_REMAPPING,
 MANAGER_ID_TO_MANAGER_CATEGORY,
 MANAGER_CATEGORY_TO_MANAGER_ID,
 TRAIN_DATAFRAME,
 TEST_DATAFRAME) = map_categorical_column_to_category_ids(
    TRAIN_DATAFRAME,
    TEST_DATAFRAME,
    'manager_id',
    'manager_id_category',
    min_freq=40
)

### 2.4AA) Parsing and Separating Out Address Components
Some properties might be in the same neighbourhood, the same street or
part of the same building. If we separate out the address components then
we might be able to get some more meaningful feature groupings.

We first parse all the components into their own columns and then map them into categories (dropping them later on).

In [16]:
import imp
import utils.dataframe
imp.reload(utils.dataframe)

TRAIN_DATAFRAME, TEST_DATAFRAME = utils.dataframe.parse_address_components(
    TRAIN_DATAFRAME,
    TEST_DATAFRAME,
    [
        "display_address",
        "street_address"
    ]
)

In [17]:
(DISP_ADDR_ID_UNKNOWN_REMAPPING,
 DISP_ADDR_TO_DISP_ADDR_CATEGORY,
 DISP_ADDR_CATEGORY_TO_DISP_ADDR_ID,
 TRAIN_DATAFRAME,
 TEST_DATAFRAME) = map_categorical_column_to_category_ids(
    TRAIN_DATAFRAME,
    TEST_DATAFRAME,
    'display_address_normalized',
    'display_address_category',
    min_freq=10
)

### 2.4AAA) Feature Engineering on Numerical Columns
Some models can't do simple math, but ratios or additions/subtractions
between things might be important. Lets do that now for all of our
numerical data

In [18]:
import imp
import utils.dataframe
imp.reload(utils.dataframe)

NUMERICAL_COLUMNS = [
    'bathrooms',
    'bedrooms',
    'price',
    'latitude',
    'longitude'
]

TRAIN_DATAFRAME, TEST_DATAFRAME = utils.dataframe.numerical_feature_engineering(
    TRAIN_DATAFRAME,
    TEST_DATAFRAME,
    NUMERICAL_COLUMNS
)

### 2.4AB) Counting Number of Photos
The number of photos a place has might be predictive of its interest as well, so lets at least count the number of photos.

In [19]:
import utils.dataframe
import imp
imp.reload(utils.dataframe)

TRAIN_DATAFRAME, TEST_DATAFRAME = utils.dataframe.count_json(
    TRAIN_DATAFRAME,
    TEST_DATAFRAME,
    "photos"
)

### 2.4A) Generating document vectors for description
The text in the descriptions are pretty messy. We can generate some document vectors to embed the entire thing into vector space.

To do that we use an LSTM to train some embeddings, similar to the Tweet2Vec paper.

In [20]:
import imp
import utils.doc2vec
import utils.dataframe
imp.reload(utils.doc2vec)
imp.reload(utils.dataframe)
TRAIN_DATAFRAME, TEST_DATAFRAME = utils.dataframe.remap_columns_with_transform(
    TRAIN_DATAFRAME,
    TEST_DATAFRAME,
    'description',
    'clean_description',
    utils.dataframe.normalize_description
)

In [ ]:
TRAIN_DATAFRAME, TEST_DATAFRAME = utils.doc2vec.column_to_doc_vectors(
    TRAIN_DATAFRAME,
    TEST_DATAFRAME,
    'clean_description',
    'label_interest_level',
    'description_vector',
    epochs=1000,
    parameters=200,
    learning_rate=0.01,
    save='word_embedding.npy',
    batch_size=100
)

### 2.5) Drop unnecessary columns

Now that we have made our data nicer to work with, we can drop all the inconvenient to work with columns.

In [21]:
DROP_COLUMNS = [
    'id',
    'created',
    'building_id',
    'clean_description',
    'description',
    'features',
    'display_address',
    'display_address_normalized',
    # We keep listing_id in the dataframe
    # since we'll need it later
    # 'listing_id',
    'manager_id',
    'photos',
    'street_address',
    'street_address_normalized',
    'interest_level',
]
TRAIN_DATAFRAME = TRAIN_DATAFRAME.drop(DROP_COLUMNS, axis=1)
# TEST_DATAFRAME doesn't have interest_level, so we remove it
# from the DROP_COLUMNS
TEST_DATAFRAME = TEST_DATAFRAME.drop(DROP_COLUMNS[:-1], axis=1)

In [22]:
TRAIN_DATAFRAME.head(5)

,bathrooms,bedrooms,latitude,listing_id,longitude,price,features_kitchen,features_room,features_home,features_fee,...,price_sub_longitude,price_mul_longitude,price_div_longitude,price_exp_longitude,latitude_add_longitude,latitude_sub_longitude,latitude_mul_longitude,latitude_div_longitude,latitude_exp_longitude,photos_count
0,1.0,1,40.7108,7170325,-73.9539,2400,0,1,0,0,...,2473.9539,-177489.3600,-32.452650,1.047601e-250,-33.2431,114.6647,-3010.722432,-0.550489,9.030140e-120,12
1,1.0,2,40.7513,7092344,-73.9722,3800,0,0,0,1,...,3873.9722,-281094.3600,-51.370650,1.568671e-265,-33.2209,114.7235,-3014.463314,-0.550900,7.839590e-120,6
2,1.0,2,40.7575,7158677,-73.9625,3495,0,0,0,0,...,3568.9625,-258498.9375,-47.253676,8.275380e-263,-33.2050,114.7200,-3014.526594,-0.551056,8.035724e-120,6
3,1.5,3,40.7145,7211212,-73.9425,3000,0,0,0,0,...,3073.9425,-221827.5000,-40.572066,7.815587e-258,-33.2280,114.6570,-3010.531916,-0.550624,9.356787e-120,5
4,1.0,0,40.7439,7225292,-73.9743,2795,0,0,0,0,...,2868.9743,-206758.1685,-37.783392,1.138460e-255,-33.2304,114.7182,-3014.001482,-0.550785,7.883996e-120,4


## 3) Fitting models

Now we can try out a few models and see what works well for the data that
we have so far.

In [18]:
CATEGORICAL_COLUMNS = ('building_id_category', 'manager_id_category')

In [19]:
(LINEAR_MODEL_STATISTICS,
 LINEAR_MODEL_LABELS,
 LINEAR_MODEL_PREDICTIONS,
 LINEAR_MODEL) = test_model_with_k_fold_cross_validation(
    basic_logistic_regression_pipeline(CATEGORICAL_COLUMNS),
    TRAIN_DATAFRAME.drop(['listing_id', 'label_interest_level'], axis=1),
    TRAIN_DATAFRAME['label_interest_level'],
    {
        "mse_loss": mean_squared_error,
        "accuracy": prediction_accuracy
    },
    n_splits=2
)

print('Linear Model', format_statistics(LINEAR_MODEL_STATISTICS))

/Users/smspillaz/Applications/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/smspillaz/Applications/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/smspillaz/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/smspillaz/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/sms

Fold 0 mse_loss: 0.41, accuracy: 0.72


/Users/smspillaz/Applications/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/smspillaz/Applications/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/smspillaz/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/smspillaz/Applications/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/sms

Fold 1 mse_loss: 0.41, accuracy: 0.72
Linear Model mse_loss: 0.41, accuracy: 0.72


## 4) Generate Submission

In [23]:
table = get_prediction_probabilities_with_columns(LINEAR_MODEL,
                                                  TEST_DATAFRAME.drop('label_interest_level', axis=1),
                                                  ['listing_id'])
table.columns = ['listing_id', 'high', 'medium', 'low']

/Users/smspillaz/Applications/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:381: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [40]:
table.to_csv('submission.csv', columns=['listing_id', 'high', 'medium', 'low'], index=False)